In [41]:
import pyreadr
from conllu import parse
import pandas as pd
import numpy as np
from io import open
from conllu import parse_incr


In [42]:
data_file_dev = open("it_partut-ud-dev.conllu", "r", encoding="utf-8")
data_file_train = open("it_partut-ud-train.conllu", "r", encoding="utf-8")
data_file_test = open("it_partut-ud-test.conllu", "r", encoding="utf-8")
sentences = []
words_lemmas = []

for tokenlist in parse_incr(data_file_dev):
    w_l = []
    for token in tokenlist:
        w_l = [token['form'], token['lemma'], token['upos']]
        words_lemmas.append(w_l)
        w_l = []
    sentences.append(tokenlist.metadata['text'])
    

for tokenlist in parse_incr(data_file_train):
    w_l = []
    for token in tokenlist:
        w_l = [token['form'], token['lemma'], token['upos']]
        words_lemmas.append(w_l)
        w_l = []
    sentences.append(tokenlist.metadata['text'])


    
for tokenlist in parse_incr(data_file_test):
    w_l = []
    for token in tokenlist:
        w_l = [token['form'], token['lemma'], token['upos']]
        words_lemmas.append(w_l)
        w_l = []
    sentences.append(tokenlist.metadata['text'])



    
        

In [43]:
words = set([w_l[0] for w_l in words_lemmas])
lemmas = set([w_l[1] for w_l in words_lemmas])
pos_tags = set([w_l[2] for w_l in words_lemmas])

# lower all
words = set([w.lower() for w in words])
lemmas = set([l.lower() for l in lemmas])
sentences = [s.lower() for s in sentences]
pos_tags = set([pt.lower() for pt in pos_tags])


In [44]:
# create a dictionary with the words as keys and the lemmas as values
words_lemmas_dict = {}
words_tag_dict = {}
for w_l in words_lemmas:
    words_lemmas_dict[w_l[0]] = w_l[1]

for w_l in words_lemmas:
    words_tag_dict[w_l[0]] = w_l[2]
    

# create dataframe from the two dictionaries
df = pd.DataFrame(list(words_tag_dict.items()), columns=['word', 'tag'])
df2 = pd.DataFrame(list(words_lemmas_dict.items()), columns=['word', 'lem'])

# merge the two dataframes
df = pd.merge(df, df2, on='word')


In [45]:
# write df content to csv
df = df.drop_duplicates()
df = df.sort_values(by=['word'])
df['word'] = df['word'].str.lower()
df['lem'] = df['lem'].str.lower()
df['tag'] = df['tag'].str.lower()

df.to_csv('./words_lemmas_tags_partut.csv', sep="\t", escapechar="\t", index=False)

In [46]:
# load dev.csv in a list
dev = pd.read_csv('./dev.csv', sep='\t', header=0)

test = pd.read_csv('./test.csv', sep='\t', header=0)

# load words_lemmas_tags_partut.csv in a list
new = pd.read_csv('./words_lemmas_tags_partut.csv', sep='\t', header=0)

#delete the first row
new = new.drop([0])
dev = dev.drop([0])
test = test.drop([0])

# transform to lower all
new['word'] = new['word'].str.lower()
new['tag'] = new['tag'].str.lower()
new['lem'] = new['lem'].str.lower()
dev['word'] = dev['word'].str.lower()
dev['tag'] = dev['tag'].str.lower()
dev['lem'] = dev['lem'].str.lower()
test['word'] = test['word'].str.lower()
test['tag'] = test['tag'].str.lower()
test['lem'] = test['lem'].str.lower()

#delete nan from dev
dev = dev.dropna()
test = test.dropna()
new = new.dropna()
        

In [47]:
# read dev.csv and take all pos tag
dev_pos = dev['tag'].tolist()
test_pos = test['tag'].tolist()

# remove duplicates
dev_pos = list(dict.fromkeys(dev_pos))
test_pos = list(dict.fromkeys(test_pos))

# delete nan
dev_pos = [x for x in dev_pos if str(x) != 'nan']
test_pos = [x for x in test_pos if str(x) != 'nan']

# create dict where the key is dev_pos and the value is pos_tags
dict_pos = {}
for i in range(len(dev_pos)):
    for j in range(len(pos_tags)):
        #if dev_pos[i] contain a substring
        if 'v_' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'verb'
        if 'p_' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'punct'
        if dev_pos[i] == 'nn':
            dict_pos[dev_pos[i]] = 'noun'
        if 'adj' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'adj'
        if 'nn_p' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'propn'
        if 'prep_a' in dev_pos[i]:
            dict_pos[dev_pos[i]] = '_'
        if 'conj_s' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'sconj'
        if 'adv' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'adv'
        if 'conj_c' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'cconj'
        if 'pron_rel' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'pron'
        if  dev_pos[i] == 'prep':
            dict_pos[dev_pos[i]] = 'adp'
        if 'c_num' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'num'
        if 'art' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'det'
        if 'pron_ind' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'pron'
        if 'pron_per' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'pron'
        if 'pron_dim' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'pron'
        if 'pron_ies' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'pron'
        if 'pron_pos' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'pron'
        if 'int' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'X'
        if 'p_apo' in dev_pos[i]:
            dict_pos[dev_pos[i]] = 'punct'


In [48]:
# replace the pos tag in dev_new with the new pos tag
for index, row in dev.iterrows():
    row['tag'] = dict_pos[row['tag']]
    
for index, row in test.iterrows():
    row['tag'] = dict_pos[row['tag']]


In [49]:
#write on csv
dev.to_csv('./dev_new.csv', sep="\t",escapechar='\t', index=False)
test.to_csv('./test_new.csv', sep="\t",escapechar='\t', index=False)

In [50]:
# concatenate dev_new.csv with words_lemmas_tags_partut.csv without duplicates
dev_new = pd.read_csv('./dev_new.csv', sep='\t', header=0)
partut = pd.read_csv('./words_lemmas_tags_partut.csv', sep='\t', header=0)

# delete nan
dev_new = dev_new.dropna()
partut = partut.dropna()

# delete duplicates
dev_new = dev_new.drop_duplicates()
partut = partut.drop_duplicates()

# concatenate
new = pd.concat([dev_new, partut])

# delete where tag is x
new = new[new.tag != 'X']

In [51]:
# write on csv
new.to_csv('./dev_partut.csv', sep="\t",escapechar='\t', index=False)